Image Generator for Train and Validation is necessary

In [ ]:
import numpy as np
import pandas as pd
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL
import cv2 as cv
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from keras import Model
from keras.utils.vis_utils import plot_model
import pathlib
import shutil

import warnings
warnings.filterwarnings("ignore")

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#base model import
from tensorflow.keras.applications import MobileNetV2 as base_mod

Data Related Initialization (Path Defination)

In [ ]:
data_dir = 'Train_Images'
val_dir = 'Val_Images'
test_dir = 'Test_Images'

Creating Image Batches

In [ ]:
batch_size = 32
img_height = 224
img_width = 224
num_classes = 2
base_ephoc = 50 
base_ephoc_executed = base_ephoc
finetune_ephoc = 200


base_learning_rate = 0.0001
finetune_learning_rate = 0.0000001

model_name = 'Mobilenet'
limb_type = 'All_Limbs'

Train Data Generator

In [ ]:
train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255
                                                                )

#### Train and Validation Dataset Generator

In [ ]:
train_ds = train_data_generator.flow_from_directory(data_dir, 
                                            seed = 123,
                                            target_size=(img_height,img_width),
                                            batch_size=batch_size,
                                            class_mode='sparse',
                                            classes=['Intact','Fractured']
                                            )

In [ ]:
val_ds = train_data_generator.flow_from_directory(val_dir, 
                                            seed = 123,
                                            target_size=(img_height,img_width),
                                            batch_size=batch_size,
                                            class_mode='sparse',
                                            classes=['Intact','Fractured']
                                        )
class_indices = train_ds.class_indices


#### Callbacks and Stopping Point defination for the model

In [ ]:
#define the model checkpoint path
model_checkpoint_path = os.path.join('Models', model_name, model_name + '_checkpoint/')

#check for the folder path, if exists skip else create the path
if not (os.path.exists(model_checkpoint_path)):
    os.makedirs(model_checkpoint_path)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
stopping_point = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=15,
    verbose=1,
    mode="max",
    restore_best_weights=True,
    start_from_epoch=1
)

Learning Rate scheduler, reschedule the learning rate if the same validation accuracy is repeated for 5 times, the lowest learning rate is also set

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              patience=5, min_lr=finetune_learning_rate)

Tensorboard Logs

In [ ]:
tensorboard_logs = tf.keras.callbacks.TensorBoard(log_dir="./tb_logs")

Data logger to retain the ephocs, train, and validation metrices during training

In [ ]:
#check for the file for saving heatmap
csv_save_path = 'Graphs/' + model_name
csv_save_file = csv_save_path + '/' + model_name + '_' + limb_type +'_train_data.csv'
csv_save_file2 = csv_save_path + '/' + model_name + '_' + limb_type +'_fine_data.csv'

if not (os.path.exists(csv_save_path)):
    os.makedirs(csv_save_path)

data_logger = tf.keras.callbacks.CSVLogger(csv_save_file, 
                                            separator= ',',
                                            append=True
                                            )
data_logger2 = tf.keras.callbacks.CSVLogger(csv_save_file2, 
                                            separator= ',',
                                            append=True
                                            )

Defining the Base Model

In [ ]:
base_model = base_mod(input_shape=(img_height,img_width,3), include_top=False, weights='imagenet')
base_model.trainable = False

Get last layer from base_model and set that as the input for the layers to be stacked on top of the base layer

In [ ]:
last_base_layer = base_model.get_layer('out_relu')
last_layer_output = last_base_layer.output
added_layer = tf.keras.layers.GlobalAveragePooling2D()(last_layer_output)
added_layer = tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None)(added_layer)
added_layer = tf.keras.layers.Dense(1024, activation='relu')(added_layer)
added_layer = tf.keras.layers.Dropout(0.4, noise_shape=None, seed=None)(added_layer)
added_layer = tf.keras.layers.Dense(1024, activation='relu')(added_layer)
added_layer = tf.keras.layers.Dropout(0.4, noise_shape=None, seed=None)(added_layer)
added_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(added_layer)

Assembling the Final Model

In [ ]:
model = tf.keras.Model(base_model.input, added_layer)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=base_learning_rate),
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics = ['accuracy'])

Draw the diagram of the model

In [ ]:
#tf.keras.utils.plot_model(model, to_file='Graphs/efficient.jpg', show_shapes=True, show_layer_names=True)

Training the Model

In [ ]:
epochs = base_ephoc  
history_base = model.fit(train_ds, validation_data=val_ds, epochs=base_ephoc, callbacks=[model_checkpoint_callback, stopping_point, data_logger, reduce_lr, tensorboard_logs])
base_ephoc_executed = len(history_base.history['loss'])

### Visualizing the Model Performance

In [ ]:
def plot_accuracy(acc,val_acc,type):
    #plot for the accuracy
    plt.figure(figsize=(16,8))
    plt.plot(acc,label='Training Accuracy')
    plt.plot(val_acc,label='Validation Accuracy')
    #plt.plot([base_ephoc_executed-1, base_ephoc_executed-1], plt.ylim(), label='Start Fine Tuning')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    #plt.show()

    #define the model checkpoint path
    figure_save_path = 'Graphs/' + model_name
    figure_save_name =  figure_save_path + '/' + model_name + '_' + limb_type + type +'_accuracy.jpg'

    #check for the folder path, if exists skip else create the path
    if not(os.path.exists(figure_save_path)):
        os.makedirs(figure_save_path)

    plt.savefig(figure_save_name, dpi=200)
    


In [ ]:
def plot_loss(loss,val_loss,type):
    #plotting the data from ephoc to
    plt.figure(figsize=(16,8))
    plt.plot(loss,label='Training Loss')
    plt.plot(val_loss,label='Validation Loss')
    #plt.plot([base_ephoc_executed-1, base_ephoc_executed-1], plt.ylim(), label='Start Fine Tuning')
    plt.legend()
    plt.title('Training and Validation Loss')
    #plt.show()

    #define the model checkpoint path
    figure_save_path = 'Graphs/' + model_name
    figure_save_name =  figure_save_path + '/' + model_name + '_' + limb_type + type +'_loss.jpg'

    #check for the folder path, if exists skip else create the path
    if not(os.path.exists(figure_save_path)):
        os.makedirs(figure_save_path)

    plt.savefig(figure_save_name, dpi=200)
    


##### Training Set

In [ ]:
#add the base acc and loss with the fintuned loss and acc to get the whole of the acc and loss
acc = history_base.history['accuracy']
val_acc = history_base.history['val_accuracy']

loss = history_base.history['loss']
val_loss = history_base.history['val_loss']

Base Plots

In [ ]:
plot_accuracy(acc,val_acc,'base')
plot_loss(loss,val_loss,'base')

## Fine Tuning the Model performance

In [ ]:
#unfreeze all the layers except batchnorm layers from block6c add layers onwards for fine tuning
for layer in model.layers[-81:-1]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=finetune_learning_rate),
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics = ['accuracy'])

In [ ]:
total_ephocs = base_ephoc_executed + finetune_ephoc

history_fine_tune = model.fit(train_ds, validation_data=val_ds, 
                                epochs=total_ephocs, 
                                initial_epoch = history_base.epoch[-1],
                                callbacks=[model_checkpoint_callback,stopping_point,data_logger2, reduce_lr, tensorboard_logs])

#### Visualizing the model performance after the fine tuning ephocs (Without the MARKER)

For Training Set

In [ ]:
acc += history_fine_tune.history['accuracy']
val_acc += history_fine_tune.history['val_accuracy']

loss += history_fine_tune.history['loss']
val_loss += history_fine_tune.history['val_loss']


#plotting the data from ephoc to
plot_accuracy(acc,val_acc,'finetune')
plot_loss(loss,val_loss,'finetune')

#### Save the final trained model

In [ ]:
#define the model checkpoint path
model_save_path = os.path.join('Models/' + model_name)

#check for the folder path, if exists skip else create the path
if not (os.path.exists(model_save_path)):
    os.makedirs(model_save_path)

#create a path for model name
model_save_file = model_save_path + '/' + model_name + '_' + limb_type +'.h5'

if os.path.exists(model_save_file):
    pass

model.save(model_save_file)

#### Making Pridiction Using the Model in Test Set

In [ ]:
model = keras.models.load_model(model_save_file)
class_subset = sorted(os.listdir('Test_Images'), reverse=True)


In [ ]:
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255
                                                                    )

In [ ]:
test_ds = test_data_generator.flow_from_directory(test_dir,
                                             target_size=(img_height, img_width),
                                             batch_size=1,
                                             class_mode='sparse',
                                             shuffle=False,
                                             classes=['Intact','Fractured'],
                                             seed=123)

True Classes of the Images

In [ ]:
true_classes = test_ds.classes
class_indices = train_ds.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())


In [ ]:
prediction = model.predict(test_ds)
pred_classes = np.argmax(prediction, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, auc, classification_report, confusion_matrix, balanced_accuracy_score, roc_curve
import seaborn as sns

#calculating each of the metrices
accuracy = accuracy_score(true_classes, pred_classes)
balanced_accuracy = balanced_accuracy_score(true_classes, pred_classes)
precision = precision_score(true_classes, pred_classes)
recall = recall_score(true_classes, pred_classes)
f1 = f1_score(true_classes, pred_classes)
auc_mod = roc_curve(true_classes, pred_classes)

Confusion Matrix, Accuracy Scores, Classification Reports and other stuffs

In [ ]:
conf_mat = confusion_matrix(true_classes, pred_classes)
fig, ax = plt.subplots(figsize=(6,6))
ax = sns.heatmap(conf_mat, annot=True, cbar=False, square=True, fmt='d', cmap=plt.cm.Blues, xticklabels=class_subset, yticklabels=class_subset,annot_kws={'size': 18})
heatmap = ax.get_figure()

#check for the file for saving heatmap
figure_save_path = 'Graphs/' + model_name + '/'
figure_save_name = figure_save_path + model_name + '_' + limb_type +'_fined_tuned_confusion_matrix.jpg'

#check for the folder path, if exists skip else create the path
if not (os.path.exists(figure_save_path)):
    os.makedirs(figure_save_path)

heatmap.savefig(figure_save_name, dpi=200)

Printing the metrices

In [ ]:
print("Model Accuracy For Given Model on Test Datset: {:.2f}%".format(accuracy * 100))
print("Balanced Accuracy For Given Model on Test Datset: {:.2f}%".format(balanced_accuracy * 100))
print("Precision For Given Model on Test Datset: {:.2f}%".format(precision * 100))
print("Recall For Given Model on Test Datset: {:.2f}%".format(recall * 100))
print("F1-Score For Given Model on Test Datset: {:.2f}%".format(f1 * 100))


#dictionary of the metrices
metrices = {'Accuracy' : [accuracy], 'Balanced_Accuracy': [balanced_accuracy], 'Precision': [precision], 'Recall': [recall], 'F1_Score': [f1], 'AUC': [auc_mod]}

#create a dataframe of the metrices
df = pd.DataFrame.from_dict(metrices, orient='columns', dtype=None, columns=None)

#check for the file for saving metrices
file_save_path = 'Graphs/' + model_name + '/'
file_save_name = file_save_path + model_name + '_' + limb_type +'_metrices.csv'

#check for the folder path, if exists skip else create the path
if not (os.path.exists(file_save_path)):
    os.makedirs(file_save_path)

df.to_csv(file_save_name, index=None)


Classification Report

In [ ]:
import pandas as pd
classi_report = classification_report(true_classes, pred_classes, output_dict=True)

classfication_report_df = pd.DataFrame(classi_report).transpose()


#check for the file for saving heatmap
file_save_path = 'Graphs/' + model_name + '/'
file_save_name = file_save_path + model_name + '_' + limb_type +'_Classification_Report.csv'

#check for the folder path, if exists skip else create the path
if not (os.path.exists(file_save_path)):
    os.makedirs(file_save_path)

classfication_report_df.to_csv(file_save_name)